In [1]:
import pandas as pd
import yfinance as yf
import json

In [3]:
tickerM = "SPY"
tickerS = "NVDA"
spy = yf.Ticker(tickerM)
nvda = yf.Ticker(tickerS)

In [4]:
print(dir(spy))
print(spy.history(period="1mo").head())

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__firstlineno__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__static_attributes__', '__str__', '__subclasshook__', '__weakref__', '_analysis', '_data', '_download_options', '_earnings', '_earnings_dates', '_expirations', '_fast_info', '_fetch_ticker_tz', '_financials', '_fundamentals', '_funds_data', '_get_ticker_tz', '_holders', '_isin', '_lazy_load_price_history', '_news', '_options2df', '_price_history', '_quote', '_shares', '_tz', '_underlying', 'actions', 'analyst_price_targets', 'balance_sheet', 'balancesheet', 'basic_info', 'calendar', 'capital_gains', 'cash_flow', 'cashflow', 'dividends', 'earnings', 'earnings_dates', 'earnings_estimate', 'earnings_history', 'eps_revisions', 'eps_trend', 'fast_info', 'financ

In [7]:
print(json.dumps(spy.info, indent=4))

{
    "longBusinessSummary": "The trust seeks to achieve its investment objective by holding a portfolio of the common stocks that are included in the index, with the weight of each stock in the portfolio substantially corresponding to the weight of such stock in the index.",
    "companyOfficers": [],
    "executiveTeam": [],
    "maxAge": 86400,
    "priceHint": 2,
    "previousClose": 563.794,
    "open": 559.28,
    "dayLow": 558.03,
    "dayHigh": 564.89,
    "regularMarketPreviousClose": 563.794,
    "regularMarketOpen": 559.28,
    "regularMarketDayLow": 558.03,
    "regularMarketDayHigh": 564.89,
    "trailingPE": 24.992039,
    "volume": 83467169,
    "regularMarketVolume": 83467169,
    "averageVolume": 55270543,
    "averageVolume10days": 72198770,
    "averageDailyVolume10Day": 72198770,
    "bid": 564.91,
    "ask": 564.97,
    "bidSize": 10,
    "askSize": 11,
    "yield": 0.0119,
    "totalAssets": 634082230272,
    "fiftyTwoWeekLow": 493.86,
    "fiftyTwoWeekHigh": 613.